In [52]:
from bs4 import BeautifulSoup
import requests
import re
import random
from itertools import chain
import collections
import pickle
from tqdm import tqdm

In [15]:
URLs = ['https://en.wikipedia.org/wiki/List_of_animal_names',
      'https://en.wikipedia.org/wiki/List_of_fish_common_names',
      'https://en.wikipedia.org/wiki/List_of_trees_and_shrubs_by_taxonomic_family',
      'https://en.wikipedia.org/wiki/List_of_birds_by_common_name']

WikiLinks = []

for URL in tqdm(URLs):

    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')

    # Find all wikiparts
    WikiRefs = soup.find_all('a')
    # Create links 
    WikiLinks = ['https://en.wikipedia.org' + pages.get('href') for pages in WikiRefs 
                           if pages.get('href') != None 
                           if pages.get('href').startswith('/wiki/')]
            
# Remove duplicates            
WikiLinks = list(set(WikiLinks))

100%|██████████| 4/4 [00:03<00:00,  1.31it/s]


In [3]:
WikiSubLinks = []

for WikiLink in tqdm(WikiLinks):

    URL = WikiLink
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')

    # Find all wikiparts
    WikiRefs = soup.find_all('a')
    # Create links 
    WikiSubLinks += ['https://en.wikipedia.org' + pages.get('href') for pages in WikiRefs 
                           if pages.get('href') != None 
                           if pages.get('href').startswith('/wiki/')]

# Drop doubles
WikiSubLinks = list(set(WikiSubLinks))

WikiSubLinks += WikiLinks
# Remove duplicates            
WikiSubLinks = list(set(WikiSubLinks))


with open('../data/processed/intermediate_wiki_links.pkl', 'wb') as f:
    pickle.dump(WikiLinks, f)

100%|██████████| 11005/11005 [1:16:14<00:00,  2.41it/s]


127985


127985

In [43]:
# Init Dict
Data = collections.defaultdict(list)

# Known chapter titles
KnownDescriptionList = ['Physical characteristics', 'Description and morphology',
                        'Description', 'General description',
                        'Appearance',
                        'Characteristics']

# Lower them
KnownDescriptionListLower =[i.lower() for i in KnownDescriptionList]

# Removes references in text
ReferenceRemover = '\[\d*\]'

for WikiLink in tqdm(WikiSubLinks):
    
    URL = WikiLink

    try:
        Response = requests.get(URL, timeout=5)
        Soup = BeautifulSoup(Response.content, "html.parser")

        # Check if the link contains description data
        Contents = [i.text.replace('[edit]', '') for i in Soup.find_all('h2')]
        if all(s not in Contents for s in KnownDescriptionList) == False:

            # Find Chapters
            for Tag in Soup.find_all('h2')[1:]:


                # Get species name
                Species = Soup.title.string\
                                    .split(' - ')[0]\
                                    .rstrip(' ')

                # Clean Chapter
                Chapter = Tag.text.strip().replace('[edit]', '')

                # Find all Text
                for Item in Tag.find_next_siblings('p'):
                    # Check if Chapter is Description or similar
                    if Chapter.lower() in KnownDescriptionListLower:
                        # Check if text belongs to current chapter
                        if Chapter in Item.find_previous_siblings('h2')[0].text.strip():
                            # Clean Text
                            CleanText = re.sub(ReferenceRemover, '', Item.text)
                            # Add to dict
                            Data[Species].append(tuple([1, CleanText]))

                    else:
                        # Check if text belongs to current chapter
                        if Chapter in Item.find_previous_siblings('h2')[0].text.strip():
                            # Clean Text
                            CleanText = re.sub(ReferenceRemover, '', Item.text)
                            # Add to dict
                            Data[Species].append(tuple([0, CleanText]))
        
        # Non species part
        else:
            try:
                # Find Chapters
                for Tag in Soup.find_all('h2')[1:]:
                    # Clean Chapter
                    Chapter = Tag.text.strip().replace('[edit]', '')
                    # Find all Text
                    for Item in Tag.find_next_siblings('p'):
                        # Check if text belongs to current chapter
                        if Chapter in Item.find_previous_siblings('h2')[0].text.strip():
                            #print(Chapter)
                            # Clean Text
                            CleanText = re.sub(ReferenceRemover, '', Item.text)
                            # Add to dict
                            Data['no_data'].append(tuple([0, CleanText]))
            # Continue on wrong links                
            except:
                continue
        
    except:
        # Continue if timeout
        continue

with open('../data/processed/train_dataWIKI_all.pkl', 'wb') as f:
    pickle.dump(Data, f)

100%|██████████| 127985/127985 [11:33:52<00:00,  3.07it/s]   


## Reduce memory for Colab

In [77]:
import copy

In [78]:
Data = pickle.load(open('../data/processed/train_dataWIKI_all.pkl', 'rb'))

In [79]:
data_values = (list(chain.from_iterable(Data.values())))
data_values = list(set(data_values))

In [80]:
len(data_values)

687064

In [81]:
# Reduce RAM on Colab
no_data = Data['no_data']
no_data_slim = list(set(random.sample(no_data, 200000)))

data_slimmed = copy.deepcopy(Data)
data_slimmed['no_data'] = no_data_slim

In [82]:
data_values_slim = (list(chain.from_iterable(data_slimmed.values())))
data_values_slim = list(set(data_values))

In [88]:
with open('../data/processed/COLAB_train_dataWIKI_all.pkl', 'wb') as f:
    pickle.dump(data_slimmed, f)